In [ ]:
#Reference Doc: #https://jovian.com/altamashwaseem04/scraping-top-selling-games-on-steam#C2

In [ ]:
# Create a new python environment 
# Python 3.8.2

In [ ]:
#%apt update
#%apt install chromium-chromedriver --quiet
#%pip install selenium --quiet
#%pip install pandas --upgrade --quiet

In [1]:
!pip install jovian --upgrade --quiet

In [119]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import pandas as pd
import re
import time

In [120]:
def create_driver(url):
  '''Takes the url as an input, creates the webdriver and returns the driver with the page'''
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome(options=options)
  wd.get(url)
  return wd

In [121]:
url='https://store.steampowered.com/search/?filter=topsellers'
wd=create_driver(url)

# OTHER URLS TO SCRAPE: 
# IF POSSIBLE SCRAPE THE TOTAL VISITORS/TOTAL INCOME/TOTAL GAMES SOLD from steam.com ⭐
# Total & Active Player Counts: https://store.steampowered.com/charts/ 
# Most Played: https://store.steampowered.com/charts/mostplayed

In [122]:
wd.get(url) # open the url in browser # collect response

In [123]:
wd.title # get the title of the page # Checks if data is loaded

'Steam Search'

In [124]:
def scroll_page(wd):
  '''Takes the driver as an input and simulates the scrolling to get all the games'''  
  SCROLL_PAUSE_TIME = 2

  # Get scroll height
  last_height = wd.execute_script("return document.body.scrollHeight")

  for i in range(6):
    # Scroll down to bottom
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
     # Calculate new scroll height and compare with last scroll height
    #If the new scroll height is equal to the previous scroll height, it means the page has reached the end, and the loop is terminated.
    #Otherwise, the new scroll height becomes the last scroll height, and the loop continues
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
  return

In [125]:
scroll_page(wd)

In [126]:
games_rows = wd.find_element(By.ID, 'search_resultsRows')
games = games_rows.find_elements(By.TAG_NAME,'a')
type(games)

list

In [127]:
def get_games(wd):
  '''Takes the driver as an input gives and returns a selenium web element with all the games'''
  games_rows = wd.find_element(By.ID, 'search_resultsRows')
  games = games_rows.find_elements(By.TAG_NAME,'a')
  return games

In [128]:
get_games(wd)

[<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_109")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_110")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_111")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_112")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_113")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_114")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e

In [129]:
def parse_data(games):
  '''Takes the web element "games" as input and returns a list of dictionary with the title and urls of the games'''
  #Creating an empty list
  games_list = []

  #looping through all the games
  for i in range(len(games)): 
    print(games[i])
    title =  games[i].find_element(By.CLASS_NAME, "title").text
    game_url = games[i].get_attribute('href')
    
    #storing the extracted information inside a dictionary
    my_game = {
        'title': title,
        'url': game_url,
    }

    #adding the dictionary inside the list
    games_list.append(my_game)
  return games_list

In [130]:
games_list=parse_data(games)

<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_109")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_110")>


<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_111")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_112")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_113")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_114")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_115")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", element="4C3D957B7DCD6655EEB112E3628BDACC_element_116")>
<selenium.webdriver.remote.webelement.WebElement (session="7b12963a023ac83de7964e4e1384bae2", 

In [131]:
games_list

[{'title': 'Counter-Strike: Global Offensive',
  'url': 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_7000_150_1'},
 {'title': 'Remnant II',
  'url': 'https://store.steampowered.com/app/1282100/Remnant_II/?snr=1_7_7_7000_150_1'},
 {'title': "Baldur's Gate 3",
  'url': 'https://store.steampowered.com/app/1086940/Baldurs_Gate_3/?snr=1_7_7_7000_150_1'},
 {'title': 'Ratchet & Clank: Rift Apart',
  'url': 'https://store.steampowered.com/app/1895880/Ratchet__Clank_Rift_Apart/?snr=1_7_7_7000_150_1'},
 {'title': 'Steam Deck',
  'url': 'https://store.steampowered.com/app/1675200/Steam_Deck/?snr=1_7_7_7000_150_1'},
 {'title': 'Dead by Daylight',
  'url': 'https://store.steampowered.com/app/381210/Dead_by_Daylight/?snr=1_7_7_7000_150_1'},
 {'title': 'Apex Legends™',
  'url': 'https://store.steampowered.com/app/1172470/Apex_Legends/?snr=1_7_7_7000_150_1'},
 {'title': 'Call of Duty®: Modern Warfare® II',
  'url': 'https://store.steampowered.com/app/1938090/Call_o

In [132]:
games_df= pd.DataFrame(games_list)

In [133]:
games_df.head()

,title,url
0,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...
1,Remnant II,https://store.steampowered.com/app/1282100/Rem...
2,Baldur's Gate 3,https://store.steampowered.com/app/1086940/Bal...
3,Ratchet & Clank: Rift Apart,https://store.steampowered.com/app/1895880/Rat...
4,Steam Deck,https://store.steampowered.com/app/1675200/Ste...


In [134]:
games_df.shape

(350, 2)

In [135]:
games_df.isnull().sum()

title    0
url      0
dtype: int64

In [136]:
## app id testing
def get_appid(game_url):
  app_id = re.search(r'(\d+)', game_url)
  if app_id:
        app = app_id.group() # Collects app i
  return app

In [137]:
# app id testing
get_appid('https://store.steampowered.com/app/1295920/The_Mortuary_Assistant/?snr=1_7_7_7000_150_7')

'1295920'

In [138]:
##price testing
def get_price(wd):
  try:  
    try:  
      try:
        price_tag = wd.find_element(By.CLASS_NAME, 'game_purchase_action')
        price = price_tag.find_element(By.CLASS_NAME, 'price').text.strip('$')
        
      except:
        price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
        prices = price_tag.text.strip('\n').split('$')
        price = prices
        
    except:
      price = 'not availale'

  except:
     price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
     price = price_tag.find_element(By.CLASS_NAME, 'discount_original_price').text

  return price



In [139]:
## test discounted price
  
def get_discounted(wd):
  try:
    try:  
      try:
        price_tag = wd.find_element(By.CLASS_NAME, 'game_purchase_action')
        discprice = price_tag.find_element(By.CLASS_NAME, 'price').text.strip('$')
          
      except:
        price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
        prices = price_tag.text.strip('\n').split('$')
        discprice = prices[2]
      
    except:
      discprice = 'not availale'

  except:
      price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
      discprice = price_tag.find_element(By.CLASS_NAME, 'discount_final_price')

  return discprice


In [140]:
## release date testing
def get_release(wd):
   try: 
      info_tag = wd.find_element(By.CLASS_NAME, 'glance_ctn_responsive_left')

      release = info_tag.find_element(By.CLASS_NAME, 'release_date').text.replace("RELEASE DATE:\n", "").strip()
   except:
      release = 'not available'
       
   return release


In [141]:
def check_page(wd):
 try: 
    try:
      info_tag = wd.find_element(By.CLASS_NAME, 'glance_ctn_responsive_left')
      return wd
    except:
      year_tag = wd.find_element(By.CLASS_NAME, 'agegate_birthday_selector')
      year = year_tag.find_element(By.ID, 'ageYear')
      yearDD = Select(year)
      yearDD.select_by_value('1900')
      view_button = wd.find_element(By.XPATH, '//*[@id="view_product_page_btn"]')
      view_button.click()
      time.sleep(4)
 except:
     return wd

 return wd


 
def get_price(wd):
  try:  
    try:  
      try:
        price_tag = wd.find_element(By.CLASS_NAME, 'game_purchase_action')
        price = price_tag.find_element(By.CLASS_NAME, 'price').text.strip('$')
        
      except:
        price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
        prices = price_tag.text.strip('\n').split('$')
        price = prices[1]
        
    except:
      price = 'not availale'

  except:
     price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
     price = price_tag.find_element(By.CLASS_NAME, 'discount_original_price').text

  return price


  
def get_discounted(wd):
  try:
    try:  
      try:
        price_tag = wd.find_element(By.CLASS_NAME, 'game_purchase_action')
        discprice = price_tag.find_element(By.CLASS_NAME, 'price').text.strip('$')
          
      except:
        price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
        prices = price_tag.text.strip('\n').split('$')
        discprice = prices[2]
      
    except:
      discprice = 'not availale'

  except:
      price_tag = wd.find_element(By.CLASS_NAME, 'discount_prices')
      discprice = price_tag.find_element(By.CLASS_NAME, 'discount_final_price')

  return discprice


def get_appid(game_url):
  app_id = re.search(r'(\d+)', game_url)
  if app_id:
        app = app_id.group() # Collects app id
  return app

def get_release(wd):
   try: 
      info_tag = wd.find_element(By.CLASS_NAME, 'glance_ctn_responsive_left')

      release = info_tag.find_element(By.CLASS_NAME, 'release_date').text.strip('RELEASE DATE:\n')
   except:
      release = 'not available'
       
   return release



#def get_reviews(wd):
 # try:  
  #  info_tag = wd.find_element(By.CLASS_NAME, 'glance_ctn_responsive_left')

   # reviews_tag = info_tag.find_element(By.CLASS_NAME, 'user_reviews').text.strip('\n')
  #except:
   # reviews_tag = 'not available'
  #return reviews_tag



def get_page(url, wd):
  wd.get(url)
  return wd




def get_game_info(url):
  wd_1 = get_page(url,wd)
  wd_new = check_page(wd_1)
  price = get_price(wd_new)
  discounted = get_discounted(wd_new)
  release = get_release(wd_new)
  appid=get_appid(url)
  #reviews = get_reviews(wd_new)
  mygame = {
            'Price': price,
            'Discounted Price': discounted,
            'Release Date': release,
            'App ID': appid
            #'Reviews': reviews
           }
  return mygame

In [142]:
def get_all_games(new_gamesdf):
  games_url = new_gamesdf['url']
  game_info_list = []
  for i in games_url:
    game_info = get_game_info(i)
    game_info_list.append(game_info)
  game_info_df = pd.DataFrame(game_info_list)
  return game_info_df

In [143]:
game_info=get_all_games(games_df)

: 

In [ ]:
game_info.head()

,Price,Discounted Price,Release Date,App ID
0,49.99,49.99,"Jul 25, 2023",1282100
1,Free to Play,Free to Play,"ug 21, 2012",730
2,59.99,59.99,"Oct 6, 2020",1086940
3,59.99,59.99,"Jul 26, 2023",1895880
4,not availale,not availale,not available,1675200


In [ ]:
merged_games_df = pd.concat([games_df, game_info], axis=1, join='inner')

In [ ]:
merged_games_df.reset_index(drop=True, inplace=True)

In [ ]:
merged_games_df.head()

,title,url,Price,Discounted Price,Release Date,App ID
0,Remnant II,https://store.steampowered.com/app/1282100/Rem...,49.99,49.99,"Jul 25, 2023",1282100
1,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,Free to Play,Free to Play,"ug 21, 2012",730
2,Baldur's Gate 3,https://store.steampowered.com/app/1086940/Bal...,59.99,59.99,"Oct 6, 2020",1086940
3,Ratchet & Clank: Rift Apart,https://store.steampowered.com/app/1895880/Rat...,59.99,59.99,"Jul 26, 2023",1895880
4,Steam Deck,https://store.steampowered.com/app/1675200/Ste...,not availale,not availale,not available,1675200


In [ ]:
merged_games_df.to_csv('Top_sellers.csv')

## From here do it in pyspark

In [35]:
free_games = merged_games_df[(merged_games_df['Price'] == 'Free to Play') |(merged_games_df['Price'] == 'Free To Play') | (merged_games_df['Price'] == 'Free')]


In [36]:

free_games['Discounted Price'] = free_games['Discounted Price'].str.replace('not available', '0')
free_games['Release Date'] = pd.to_datetime(free_games['Release Date'], errors='coerce')

<ipython-input-36-690942ef2899>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  free_games['Discounted Price'] = free_games['Discounted Price'].str.replace('not available', '0')
<ipython-input-36-690942ef2899>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  free_games['Release Date'] = pd.to_datetime(free_games['Release Date'], errors='coerce')


In [37]:
free_games.shape


(44, 6)

In [38]:
free_games

,title,url,Price,Discounted Price,Release Date,App ID
1,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,Free to Play,Free to Play,NaT,730
6,Apex Legends™,https://store.steampowered.com/app/1172470/Ape...,Free to Play,Free to Play,2020-11-04,1172470
11,Team Fortress 2,https://store.steampowered.com/app/440/Team_Fo...,Free to Play,Free to Play,2007-10-10,440
14,War Thunder,https://store.steampowered.com/app/236390/War_...,Free to Play,Free to Play,NaT,236390
18,Destiny 2,https://store.steampowered.com/app/1085660/Des...,Free To Play,Free To Play,2019-10-01,1085660
19,The Sims™ 4,https://store.steampowered.com/app/1222670/The...,Free To Play,Free To Play,NaT,1222670
20,Lost Ark,https://store.steampowered.com/app/1599340/Los...,Free To Play,Free To Play,2022-02-11,1599340
29,Warframe,https://store.steampowered.com/app/230410/Warf...,Free to Play,Free to Play,2013-03-25,230410
32,Phantasy Star Online 2 New Genesis,https://store.steampowered.com/app/1056640/Pha...,Free To Play,Free To Play,NaT,1056640
39,PUBG: BATTLEGROUNDS,https://store.steampowered.com/app/578080/PUBG...,Free to Play,Free to Play,NaT,578080


In [278]:
games = merged_games_df[~merged_games_df['Price'].isin(['Free to Play', 'Free To Play', 'Free','not availale'])]


In [279]:
games.shape

(320, 6)

In [280]:
games.head()


,title,url,Price,Discounted Price,Release Date,App ID
2,BattleBit Remastered,https://store.steampowered.com/app/671860/Batt...,14.99,14.99,"Jun 15, 2023",671860
3,The Legend of Heroes: Trails into Reverie,https://store.steampowered.com/app/1668540/The...,59.99,not availale,"Jul 7, 2023",1668540
4,Baldur's Gate 3,https://store.steampowered.com/app/1086940/Bal...,59.99,59.99,"Oct 6, 2020",1086940
5,DAVE THE DIVER,https://store.steampowered.com/app/1868140/DAV...,19.99\n,17.99,"Jun 28, 2023",1868140
7,Rust,https://store.steampowered.com/app/252490/Rust...,39.99\n,26.79,"Feb 8, 2018",252490


In [281]:
## Final df for all top sellers except free ones
games.shape

(320, 6)

In [282]:
games['Price'] = games['Price'].str.replace('\n', '')
games['Discounted Price'] = games['Discounted Price'].str.replace('\n', '')
games['Discounted Price'] = games['Discounted Price'].str.replace('not availale', '0')
games['Release Date'] = pd.to_datetime(games['Release Date'], errors='coerce')


<ipython-input-282-efb44c1df1a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['Price'] = games['Price'].str.replace('\n', '')
<ipython-input-282-efb44c1df1a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['Discounted Price'] = games['Discounted Price'].str.replace('\n', '')
<ipython-input-282-efb44c1df1a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [283]:

games.head()

,title,url,Price,Discounted Price,Release Date,App ID
2,BattleBit Remastered,https://store.steampowered.com/app/671860/Batt...,14.99,14.99,2023-06-15,671860
3,The Legend of Heroes: Trails into Reverie,https://store.steampowered.com/app/1668540/The...,59.99,0,2023-07-07,1668540
4,Baldur's Gate 3,https://store.steampowered.com/app/1086940/Bal...,59.99,59.99,2020-10-06,1086940
5,DAVE THE DIVER,https://store.steampowered.com/app/1868140/DAV...,19.99,17.99,2023-06-28,1868140
7,Rust,https://store.steampowered.com/app/252490/Rust...,39.99,26.79,2018-02-08,252490


In [284]:
## save to csv files
## all games except free ones
games.to_csv('paid_games.csv', index=False)

In [285]:
## saving free games data
free_games.to_csv('free_games.csv', index=False)